In [39]:
import requests # to actually call the api and connect
import pandas as pd #to manipulate the data and convert to df, etc.
import os
from pprint import pprint #this will help make dictionaries more readable

In [15]:
# get the url to the api and define the keys that identify content within api
url = "https://sportsbook-api2.p.rapidapi.com/v0/advantages/?type=ARBITRAGE"
Headers = {'x-rapidapi-host': 'sportsbook-api2.p.rapidapi.com',
          'x-rapidapi-key': 'e979ffdb07msh206666146a5693fp12fe26jsn6dc26b3d39d1'}
response = requests.get(url, headers = Headers)
print(response)

ResponseData = response.json() # make sure response is 200 - meaning successful call to api

<Response [200]>


In [16]:
pprint(ResponseData.keys()) # gives us keys in the api so we can see what the data tells us - check to see if we need all keys

dict_keys(['advantages'])


In [17]:
advantages = ResponseData['advantages'] # specifically returns arbiratge data/ data we care about

pprint(advantages[0]) # inspects the first record, since there's thousands of records

{'createdAt': '2025-12-10T23:17:46.810Z',
 'key': 'advg-xxxx-0000',
 'lastFoundAt': '2025-12-10T23:17:46.810Z',
 'market': {'event': {'competitionInstance': {'competition': {'key': 'Q63E-wddv-ddp4',
                                                              'name': 'National '
                                                                      'Football '
                                                                      'League',
                                                              'shortName': 'NFL',
                                                              'sport': 'AMERICAN_FOOTBALL'},
                                              'competitionKey': 'Q63E-wddv-ddp4',
                                              'endAt': '2025-12-10T23:17:46.810Z',
                                              'key': 'k0qU-zcrx-Fger',
                                              'name': '2025-26 National '
                                                      'Football League',

In [43]:
# creating the dataframe - parsing only for the arbitrage bets, team names, and event types
# creating an empty list to store the data from the for loop
rows = []

# loop through each dictionary within the api and call atributes like who is playing and which team the odds favor, etc.
# using get because if some of the fields are not present, it will continue running the rest of my code
for adv in advantages:   
    market = adv.get("market", {})
    event = market.get("event", {})
    participants = event.get("participants", [])
    event_name = event.get("name")
    start_time = event.get("startTime")
    sport_type = market.get('event', {}) \
                       .get('competitionInstance', {}) \
                       .get('competition', {}) \
                       .get('sport', None)

    # getting all of the different bet 'outcomes', which is just both teams offering an opportunity for arbitrage betting
    for out in adv.get("outcomes", []):
        
        # go to the participant dictionary wihin the api, pull the names of the teams, the short name i.e. PHX, and the unique identifyer, stored in 'key'
        participant_dict = out.get("participant", {})
        participant_name = participant_dict.get("name", "Unknown")
        participant_short = participant_dict.get("shortName", "Unknown")
        participant_key = participant_dict.get("key", "Unknown")

        # adding rows to define the times the advantages were created, the event times, names of the participants to add to the stored rows[] from earlier
        rows.append({
            "advantage_type": adv.get("type"),
            "lastFoundAt": adv.get("lastFoundAt"),
            "event_name": event_name,
            "event_start_time": start_time,
            "market_type": market.get("type"),
            "market_segment": market.get("segment"),
            "modifier": out.get("modifier"),
            "payout": out.get("payout"),
            "source": out.get("source"),
            "live": out.get("live"),
            "participant_name": participant_name,
            "participant_shortName": participant_short,
            'sport_type': sport_type 
        })

# take all of the data appended into the 'rows[]' list and create a dataframe for easy manipulation and connection to Tableau
df = pd.DataFrame(rows)
df

# print only the first 5 rows
df.head()

,advantage_type,lastFoundAt,event_name,event_start_time,market_type,market_segment,modifier,payout,source,live,participant_name,participant_shortName,sport_type
0,ARBITRAGE,2025-12-10T23:17:46.810Z,Detroit Lions @ Los Angeles Rams,2025-12-14T21:25:00.000Z,MONEYLINE,FULL_MATCH,0.0,1.4200,BET_MGM,False,Los Angeles Rams,LAR,AMERICAN_FOOTBALL
1,ARBITRAGE,2025-12-10T23:17:46.810Z,Detroit Lions @ Los Angeles Rams,2025-12-14T21:25:00.000Z,MONEYLINE,FULL_MATCH,0.0,3.4000,BOVADA,False,Detroit Lions,DET,AMERICAN_FOOTBALL
2,ARBITRAGE,2025-12-10T23:17:46.810Z,Detroit Lions @ Los Angeles Rams,2025-12-14T21:25:00.000Z,MONEYLINE,FULL_MATCH,0.0,1.4200,BET_MGM,False,Los Angeles Rams,LAR,AMERICAN_FOOTBALL
3,ARBITRAGE,2025-12-10T23:17:46.810Z,Detroit Lions @ Los Angeles Rams,2025-12-14T21:25:00.000Z,MONEYLINE,FULL_MATCH,0.0,3.3853,KUTT,False,Detroit Lions,DET,AMERICAN_FOOTBALL
4,ARBITRAGE,2025-12-10T23:17:46.810Z,Ottawa Senators @ Columbus Blue Jackets,2025-12-12T00:00:00.000Z,POINT_SPREAD,FULL_MATCH,-1.5,3.2000,BET_PARX,False,Ottawa Senators,OTT,ICE_HOCKEY


In [44]:
# make event_start_time & lastFoundAt more readable
columns_to_edit = ['lastFoundAt', 'event_start_time']

for col in columns_to_edit:
    df[col] = pd.to_datetime(df[col])

df['lastFoundAt'] = df['lastFoundAt'].dt.strftime("%B %d, %Y at %I:%M %p")
df['event_start_time'] = df['event_start_time'].dt.strftime("%B %d, %Y at %I:%M %p")

# check to see if changes worked
df.head()

,advantage_type,lastFoundAt,event_name,event_start_time,market_type,market_segment,modifier,payout,source,live,participant_name,participant_shortName,sport_type
0,ARBITRAGE,"December 10, 2025 at 11:17 PM",Detroit Lions @ Los Angeles Rams,"December 14, 2025 at 09:25 PM",MONEYLINE,FULL_MATCH,0.0,1.4200,BET_MGM,False,Los Angeles Rams,LAR,AMERICAN_FOOTBALL
1,ARBITRAGE,"December 10, 2025 at 11:17 PM",Detroit Lions @ Los Angeles Rams,"December 14, 2025 at 09:25 PM",MONEYLINE,FULL_MATCH,0.0,3.4000,BOVADA,False,Detroit Lions,DET,AMERICAN_FOOTBALL
2,ARBITRAGE,"December 10, 2025 at 11:17 PM",Detroit Lions @ Los Angeles Rams,"December 14, 2025 at 09:25 PM",MONEYLINE,FULL_MATCH,0.0,1.4200,BET_MGM,False,Los Angeles Rams,LAR,AMERICAN_FOOTBALL
3,ARBITRAGE,"December 10, 2025 at 11:17 PM",Detroit Lions @ Los Angeles Rams,"December 14, 2025 at 09:25 PM",MONEYLINE,FULL_MATCH,0.0,3.3853,KUTT,False,Detroit Lions,DET,AMERICAN_FOOTBALL
4,ARBITRAGE,"December 10, 2025 at 11:17 PM",Ottawa Senators @ Columbus Blue Jackets,"December 12, 2025 at 12:00 AM",POINT_SPREAD,FULL_MATCH,-1.5,3.2000,BET_PARX,False,Ottawa Senators,OTT,ICE_HOCKEY


In [45]:
# convert to csv for my own personal data connection 

df.to_csv("arbitrage_data.csv", index = False)

In [46]:
# get working directory to connect file to tableau
print(os.getcwd())

C:\Users\alaou
